In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


Note: I was intending to try XGBClassifier and Tensorflow but I'm not confident enough in the theory to conduct hyperparameter tuning so I scrapped them.

## Imports

In [2]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier

import tensorflow as tf

## Data

In [3]:
train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [4]:
train_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


## Cleaning

In [5]:
def clean(train_data):
    data = train_data.copy()
    data[["Deck","Number","Side"]] = data["Cabin"].str.split("/",expand=True)
    data = data.drop(columns=["Cabin","PassengerId","Name"])
    mode = ["HomePlanet","Destination","CryoSleep","VIP","Deck","Number","Side"]
    modes = data[mode].mode().iloc[0]
    data[mode] = data[mode].fillna(modes)
    med = ["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]
    meds = data[med].median()
    data[med] = data[med].fillna(meds)
    data = pd.get_dummies(data, columns=["Destination", "HomePlanet", "Deck", "Side"], dtype=int)
    data["Spent"] = data["RoomService"]+data["FoodCourt"]+data["ShoppingMall"]+data["Spa"]+data["VRDeck"]
    ss = StandardScaler()
    sscols = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","Spent"]
    data[sscols] = ss.fit_transform(data[sscols])
    return data

In [6]:
data = clean(train_data)
y_train = data["Transported"].astype(int)
X_train = data.drop(columns=["Transported"])

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)

## kNN

In [8]:
param_grid = {
    "n_neighbors": list(range(1,11))
}
knn_grid = GridSearchCV(KNeighborsClassifier(), param_grid, n_jobs=-1)
knn_grid.fit(X_train, y_train)
print(knn_grid.best_params_)
grid_prediction = knn_grid.predict(X_valid)
print(classification_report(y_valid, grid_prediction))

{'n_neighbors': 1}
              precision    recall  f1-score   support

           0       0.57      0.55      0.56       863
           1       0.57      0.59      0.58       876

    accuracy                           0.57      1739
   macro avg       0.57      0.57      0.57      1739
weighted avg       0.57      0.57      0.57      1739



## Naive Bayes

In [9]:
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_valid)
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.59      0.70       863
           1       0.69      0.92      0.79       876

    accuracy                           0.75      1739
   macro avg       0.78      0.75      0.75      1739
weighted avg       0.78      0.75      0.75      1739



## Logistical Regression

In [10]:
param_grid = {
    "penalty":[None,"l2","l1","elasticnet"],
    "C":[0.001, 0.01, 0.1, 1, 10, 100]
}
log_grid = GridSearchCV(LogisticRegression(), param_grid, n_jobs=-1)
log_grid.fit(X_train, y_train)
print(log_grid.best_params_)
grid_prediction = log_grid.predict(X_valid)
print(classification_report(y_valid, grid_prediction))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/prepro

{'C': 0.001, 'penalty': None}
              precision    recall  f1-score   support

           0       0.80      0.79      0.79       863
           1       0.79      0.80      0.80       876

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739



/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Support Vector Machines

In [11]:
# param_grid = {
#     "kernel":["linear", "rbf"],
#     "C":[10],
#     "degree":list(range(3,5))
# }
# svm_grid = GridSearchCV(SVC(), param_grid, n_jobs=-1)
# svm_grid.fit(X_train, y_train)
# print(svm_grid.best_params_)
# grid_prediction = svm_grid.predict(X_valid)
# print(classification_report(y_valid, grid_prediction))

## Random Forest Classifiers

In [12]:
param_grid = {
    "n_estimators":[100, 200, 500],
    "criterion":["gini","entropy","log_loss"]
}   
rfc_grid = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=-1)
rfc_grid.fit(X_train, y_train)
print(rfc_grid.best_params_)
grid_prediction = rfc_grid.predict(X_valid)
print(classification_report(y_valid, grid_prediction))

{'criterion': 'entropy', 'n_estimators': 500}
              precision    recall  f1-score   support

           0       0.79      0.86      0.82       863
           1       0.85      0.77      0.81       876

    accuracy                           0.81      1739
   macro avg       0.82      0.81      0.81      1739
weighted avg       0.82      0.81      0.81      1739



## XGBoost

In [13]:
# param_grid = {
#     "max_depth": range(20,41,1),
#     "n_estimators":range(100, 501, 50),
#     "learning_rate":[0.5, 0.1,0.05,0.01],
#     "enable_categorical":[True]
# }
# boost_grid = GridSearchCV(XGBClassifier(), param_grid, scoring="roc_auc", cv=5, n_jobs=-1)
# boost_grid.fit(X_train, y_train)
# print(boost_grid.best_params_)
# grid_prediction = boost_grid.predict(X_valid)
# print(classification_report(y_valid, grid_prediction))

## Voting Classifier

In [14]:
model = VotingClassifier([
    ("1",knn_grid),
    ("2",nb),
    ("3",log_grid),
    ("4",rfc_grid),
])
model.fit(X_train, y_train)
y_pred = model.predict(X_valid)
print(classification_report(y_valid, y_pred))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/prepro

              precision    recall  f1-score   support

           0       0.78      0.83      0.81       863
           1       0.82      0.77      0.80       876

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739



## Test Data

In [15]:
passengers = test_data["PassengerId"]
X_test = clean(test_data)

In [16]:
y_test = model.predict(X_test)

In [17]:
final = pd.DataFrame(y_test, index=passengers)
final.columns=["Transported"]
final["Transported"] = final["Transported"].astype(bool)
print(final)

             Transported
PassengerId             
0013_01             True
0018_01            False
0019_01             True
0021_01             True
0023_01             True
...                  ...
9266_02             True
9269_01            False
9271_01             True
9273_01             True
9277_01             True

[4277 rows x 1 columns]


In [18]:
final.to_csv("submission.csv")